In [7]:
import pandas as pd

all_df = pd.read_csv(r'test_sale.csv').head()

,Unnamed: 0,Sale Start,Sale End,Final price,Origin Price,Sale Rate,DateTime,Ready to Sale,Sale Period,Game_ID
0,0,2015-11-25 18:24:05,2015-12-01 18:39:34,5250.0,10500.0,0.50,2015-11-25 18:24:05,21 days 00:54:41,6 days 00:15:29,10
1,1,2015-12-22 19:34:15,2016-01-04 19:21:20,5250.0,10500.0,0.50,2015-12-22 19:34:15,32 days 00:10:02,12 days 23:47:05,10
2,2,2016-02-05 19:31:22,2016-02-12 18:47:14,2620.0,10500.0,0.75,2016-02-05 19:31:22,285 days 01:47:07,6 days 23:15:52,10
3,3,2016-11-23 20:34:21,2016-12-01 16:14:00,2620.0,10500.0,0.75,2016-11-23 20:34:21,21 days 01:44:51,7 days 19:39:39,10
4,4,2016-12-22 17:58:51,2017-01-02 18:23:32,2620.0,10500.0,0.75,2016-12-22 17:58:51,171 days 02:48:31,11 days 00:24:41,10


In [4]:
import pandas as pd
from datetime import datetime

all_df = pd.read_csv(r'test_sale.csv').head()
game_info_df = pd.read_csv(r'static_data\game_info.csv')
all_sale_df = pd.read_csv('static_data\전체세일.csv').head()

game_info_df = game_info_df.drop(columns=['DLC_Game'])
#pd.to_datetime(datetime.today()) - pd.to_datetime(game_info_df['release date'])
test_day = game_info_df['release date'].iloc[0]

game_info_df['release date'] = game_info_df['release date'].str.replace(',','')
temp_index = game_info_df[game_info_df['release date'].apply(lambda x : len(x.split())) < 3].index
game_info_df['release date'].iloc[temp_index] = '1 ' + game_info_df['release date'].iloc[temp_index]

game_info_df['last Day'] = (pd.to_datetime(datetime.today()) - pd.to_datetime(game_info_df['release date'])).apply(lambda x : str(x).split()[0])
game_info_df = game_info_df.drop(columns=['Unnamed: 0','Unnamed: 0.2','Name','release date'])
game_info_df['Company'] = game_info_df['Company'].apply(lambda x : x.split(',')[0])

df = pd.read_csv(r'test_sale.csv')
df['Ready to Sale Day'] = df['Ready to Sale'].apply(lambda x : x.split()[0])
df['Sale Peroid Day'] = df['Sale Period'].apply(lambda x : int(x.split()[0])+1)

temp_df = df.drop(columns=['Unnamed: 0','Ready to Sale','Origin Price','Sale Rate','Sale Period', 'Sale Start','Sale End','DateTime'])

test_df = pd.merge(temp_df, game_info_df,left_on='Game_ID', right_on='AppID', how='right')

test_df['price'] = test_df['price'].str.extract('(\d+)')
test_df['price'] = test_df['price'].fillna(0)

test_df['price'] = test_df['price'].astype(int)
test_df['last Day'] = test_df['last Day'].astype(int)
test_df['price'] = test_df['price'].astype(int)

running_df = pd.get_dummies(test_df)

running_df = running_df.dropna()
running_df

<>:28: SyntaxWarning: invalid escape sequence '\d'
<>:28: SyntaxWarning: invalid escape sequence '\d'
C:\Users\GJ\AppData\Local\Temp\ipykernel_15044\2963675479.py:28: SyntaxWarning: invalid escape sequence '\d'
  test_df['price'] = test_df['price'].str.extract('(\d+)')
C:\Users\GJ\AppData\Local\Temp\ipykernel_15044\2963675479.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_info_df['release date'].iloc[temp_index] = '1 ' + game_info_df['release date'].iloc[temp_index]


,Final price,Game_ID,Sale Peroid Day,AppID,price,latest rating counter,all rating counter,last Day,Ready to Sale Day_0,Ready to Sale Day_1,...,"category_['Racing', 'Simulation', 'Early Access']","category_['Racing', 'Sports']",category_['Racing'],"category_['Simulation', 'Early Access']","category_['Simulation', 'Sports', 'Strategy']","category_['Simulation', 'Sports']","category_['Simulation', 'Strategy']",category_['Simulation'],category_['Strategy'],category_['Utilities']
0,5250.0,10.0,7.0,10,10500,1278,146415,8491,False,False,...,False,False,False,False,False,False,False,False,False,False
1,5250.0,10.0,13.0,10,10500,1278,146415,8491,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2620.0,10.0,7.0,10,10500,1278,146415,8491,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2620.0,10.0,8.0,10,10500,1278,146415,8491,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2620.0,10.0,12.0,10,10500,1278,146415,8491,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12492,3300.0,975410.0,8.0,975410,5500,0,29,1888,False,False,...,False,False,False,False,False,False,False,False,False,False
12493,3300.0,975410.0,15.0,975410,5500,0,29,1888,False,False,...,False,False,False,False,False,False,False,False,False,False
12494,2200.0,975410.0,1.0,975410,5500,0,29,1888,True,False,...,False,False,False,False,False,False,False,False,False,False
12495,2200.0,975410.0,14.0,975410,5500,0,29,1888,False,False,...,False,False,False,False,False,False,False,False,False,False


In [2]:
running_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12487 entries, 0 to 12496
Columns: 852 entries, Final price to category_['Utilities']
dtypes: bool(844), float64(3), int32(2), int64(3)
memory usage: 10.8 MB


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import pickle


x = running_df.drop(['price'],axis=1)
y = running_df['price']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=111)

# sklearn 에 적재되어 있는 알고리즘
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# 설치가 필요한 앙상블 알고리즘 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 평가함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# 분류 기본 알고리즘
tree = DecisionTreeClassifier()     # 의사결정트리
logistic = LogisticRegression()     # 로지스틱회귀
svc = SVC(probability=True)         # SVC
knn = KNeighborsClassifier()        # KMean
nb = GaussianNB()
#cat = CatBoostClassifier()

# Ensemble
rfc = RandomForestClassifier(n_estimators=1000)         # 랜덤 포래스트
bag = BaggingClassifier(n_estimators=100)               # bagging
ada = AdaBoostClassifier(n_estimators=500)              # AdaBoost
grad = GradientBoostingClassifier(n_estimators=500)     # GradientBoost
xgb = XGBClassifier(n_estimators=500)                   # xgbBoost
lgbm = LGBMClassifier()                                 # lightgbBoost

models = [ tree,logistic,svc, knn, nb, rfc, bag, ada, grad, xgb, lgbm ]

#models = [rfc,xgb,lgbm]
check_df = pd.DataFrame(columns=['model','score','time'])

data_model = dict()
data_confirm = dict()
data_preds = dict()
from sklearn.metrics import classification_report


model_path = 'ml_model'

# 1차 검색 테스트
for model in models:
    print('{} model 시작'.format(model))

    try:
        timer = time.time()
        model.fit(x_train,y_train)
        preds = model.predict(x_test)
        score = accuracy_score(y_test, preds)
        check_df.loc[len(check_df)] = [model.__class__.__name__, score, round(timer,3)]
        data_model[model.__class__.__name__] = model
        data_confirm[model.__class__.__name__] = classification_report(y_test, preds)
        data_preds[model.__class__.__name__] = preds

        with open(os.path.join(model_path,'{}.pickle'.format(model.__class__.__name__)),'wb') as f:
            pickle.dump(model, f)
    except:
        print('error model : {}'.format(model.__class__.__name__))


check_df

DecisionTreeClassifier() model 시작
LogisticRegression() model 시작


c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: Undefi

SVC(probability=True) model 시작


c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

KNeighborsClassifier() model 시작


...

c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

RandomForestClassifier(n_estimators=1000) model 시작
BaggingClassifier(n_estimators=100) model 시작
AdaBoostClassifier(n_estimators=500) model 시작


c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

GradientBoostingClassifier(n_estimators=500) model 시작
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) model 시작
error model : XGBClassifier
LGBMClassifier() model 시작
error model : LGBMClassifier


,model,score,time
0,DecisionTreeClassifier,0.994796,1.706690e+09
1,LogisticRegression,0.096477,1.706690e+09
2,SVC,0.156525,1.706690e+09
3,KNeighborsClassifier,0.984788,1.706690e+09
4,GaussianNB,0.415933,1.706690e+09
5,RandomForestClassifier,0.999199,1.706690e+09
6,BaggingClassifier,0.997598,1.706690e+09
7,AdaBoostClassifier,0.095677,1.706691e+09
8,GradientBoostingClassifier,0.998799,1.706691e+09


---

In [6]:
csv_root_path = 'ml_model'
csv_folder_list = os.listdir(csv_root_path)

file_list = []

for item in csv_folder_list:
    print(item.split('.')[0])
    

AdaBoostClassifier
BaggingClassifier
DecisionTreeClassifier
GaussianNB
GradientBoostingClassifier
KNeighborsClassifier
LogisticRegression
RandomForestClassifier
SVC


In [9]:
#for data_model
for key, value in data_confirm.items():
    print(value)

    print()

data_confirm['AdaBoostClassifier']


              precision    recall  f1-score   support

           0       0.96      0.98      0.97        50
        1100       1.00      0.97      0.98        33
        1290       1.00      1.00      1.00         1
        2200       1.00      1.00      1.00        18
        2300       1.00      1.00      1.00        70
        2700       1.00      1.00      1.00         5
        3300       0.89      0.89      0.89         9
        3400       0.71      1.00      0.83         5
        4400       1.00      1.00      1.00        47
        4500       1.00      1.00      1.00         7
        5000       1.00      1.00      1.00        20
        5500       0.98      0.98      0.98        52
        5600       1.00      0.97      0.98        58
        5900       1.00      0.95      0.98        21
        6500       1.00      1.00      1.00        39
        6700       0.50      1.00      0.67         1
        7500       1.00      1.00      1.00         7
        7800       1.00    

TypeError: type 'AdaBoostClassifier' is not subscriptable

In [ ]:


for key, value in data_preds.items():
    print(value)
#data_preds